In [1]:
from import_hack import *
from methodsnm.mesh_4d import *
from methodsnm.visualize import *
import numpy as np
m=4
mesh = StructuredTesseraktMesh(m,m,m,m)

source module for methodsNM imported.


In [2]:
from methodsnm.fes import *
fes = P1_Tesserakt_Space(mesh)

In [3]:
from methodsnm.forms import *
from methodsnm.formint import *
from numpy import pi, cos ,sin

blf = BilinearForm(fes)
c = GlobalFunction(lambda x: 1, mesh = mesh)
#blf += LaplaceIntegral_without_time(c)
blf += TimeIntegral(c)
blf.assemble()

lf = LinearForm(fes)
f = GlobalFunction(lambda x: pi*cos(pi*x[3]), mesh = mesh)
lf += SourceIntegral(f)
lf.assemble()

In [ ]:
def list_diff(a, b):
    """Entfernt alle Elemente aus Liste a, die in Liste b enthalten sind."""
    return [x for x in a if x not in b]

uh = FEFunction(fes)
f = lambda x: sin(pi*x[3])
uh._set(f, True)
res = lf.vector - blf.matrix.dot(uh.vector)

from scipy.sparse.linalg import spsolve
from methodsnm.solver import solve_on_freedofs
freedofs = list_diff(mesh.vertices,mesh.bndry_vertices)
#uh.vector = spsolve(blf.matrix, lf.vector)
uh.vector += solve_on_freedofs(blf.matrix,res,freedofs)
from methodsnm.forms import compute_difference_L2

uex =  GlobalFunction(lambda x: sin(pi*x[3]), mesh = mesh)
l2diff = compute_difference_L2(uh, uex, mesh, intorder = 5)

print("l2diff =", l2diff)


l2diff = 56966675.097956955


In [5]:
def list_diff(a, b):
    """Entfernt alle Elemente aus Liste a, die in Liste b enthalten sind."""
    return [x for x in a if x not in b]

top = mesh.filter_bndry_points("max",3)
boundary = list_diff(mesh.bndry_vertices,top)

uh = FEFunction(fes)
f = lambda x: sin(pi*x[0])*sin(pi*x[1])*sin(pi*x[2])
uh._set(f, boundary=boundary)
res = lf.vector - blf.matrix.dot(uh.vector)

from scipy.sparse.linalg import spsolve
from methodsnm.solver import solve_on_freedofs
freedofs = list_diff(mesh.vertices,boundary)
#uh.vector = spsolve(blf.matrix, lf.vector)
uh.vector = solve_on_freedofs(blf.matrix,res,freedofs)

from methodsnm.forms import compute_difference_L2

uex =  GlobalFunction(lambda x: sin(pi*x[0])*sin(pi*x[1])*sin(pi*x[2])*np.exp(-3*pi**2*x[3]), mesh = mesh)
l2diff = compute_difference_L2(uh, uex, mesh, intorder = 5)

print("l2diff =", l2diff)

l2diff = 4.2754064644713864e+16
